<a href="https://colab.research.google.com/github/itberrios/think_autonomous/blob/main/3d_reconstruction/SFM_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Structure From Motion!
This is the final step — and one of the most exciting project you can build in 3D Computer Vision!

Here's what we're going to do:

*   Load 2 or more images
*   Match Features
*   Estimate the Fundamental Matrix
*   Estimate the Essential Matrix
*   Recover R and T
*   Triangulate
*   Reconstruct in 3D



In [ ]:
import matplotlib.pyplot as plt
#%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

import cv2
import numpy as np

#auto-reloading external modules
%load_ext autoreload
%autoreload 2

In [ ]:
!wget https://stereo-vision.s3.eu-west-3.amazonaws.com/fountain.zip && unzip fountain.zip

### 1. Images

In [ ]:
#Reading two images for reference
img1 = cv2.imread('fountain/0001.png')
img2 = cv2.imread('fountain/0002.png')

#Converting from BGR to RGB format
img1 = img1[:,:,::-1]
img2 = img2[:,:,::-1]

#NOTE: you can adjust appropriate figure size according to the size of your screen
f, (ax0, ax1) = plt.subplots(1,2,figsize=(9,4))
ax0.imshow(img1)
ax1.imshow(img2)
plt.show()

### 2. Feature Matching

In [ ]:
sift = cv2.SIFT_create()
kp = sift.detect(img2,None)

In [ ]:
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

In [ ]:
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50) # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

In [ ]:
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
 if m.distance < 0.7*n.distance:
  matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
 singlePointColor = (255,0,0),
 matchesMask = matchesMask,
 flags = cv2.DrawMatchesFlags_DEFAULT)
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
plt.imshow(img3,),plt.show()

In [ ]:
len(matches)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(des1.astype(np.uint8), des2.astype(np.uint8))
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

In [ ]:
def GetImageMatches(img1,img2, match_type='bf'):
    # ref: https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    if match_type == 'bf':
      bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
      # Match descriptors.
      matches = bf.match(des1.astype(np.uint8), des2.astype(np.uint8))
      # Sort them in the order of their distance.
      matches = sorted(matches, key = lambda x:x.distance)

    # else: # use KNN matching
    #   # FLANN parameters
    #   FLANN_INDEX_KDTREE = 1
    #   index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    #   search_params = dict(checks=50) # or pass empty dictionary
    #   flann = cv2.FlannBasedMatcher(index_params,search_params)
    #   matches = flann.knnMatch(des1,des2,k=2)

    return kp1,des1,kp2,des2,matches

def GetAlignedMatches(kp1,desc1,kp2,desc2,matches):
    img1idx = np.array([m.queryIdx for m in matches])
    img2idx = np.array([m.trainIdx for m in matches])

    # filter keypoints that were not matched
    kp1_ = (np.array(kp1))[img1idx]
    kp2_ = (np.array(kp2))[img2idx]

    # retreive image coordinates of amtched keypoints
    img1pts = np.array([kp.pt for kp in kp1_])
    img2pts = np.array([kp.pt for kp in kp2_])

    return img1pts,img2pts,img1idx,img2idx

#Getting SIFT/SURF features for image matching (this might take a while)
match_type = 'bf'
kp1,desc1,kp2,desc2,matches=GetImageMatches(img1, img2, match_type)
#kp1,desc1,kp2,desc2,matches, img1pts, img2pts = feature_matching(img1, img2)

#Aligning two keypoint vectors
img1pts,img2pts,img1idx,img2idx=GetAlignedMatches(kp1, desc1, kp2, desc2, matches)

In [ ]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                    singlePointColor = None,
                    flags = 2)

plt.imshow(cv2.drawMatches(img1,kp1,img2,kp2,matches,None,**draw_params))

### 3. Estimate Fundamental Matrix

In [ ]:
F, mask = cv2.findFundamentalMat(img1pts, img2pts, cv2.FM_7POINT) # cv2.FM_RANSACcv2.FM_LMEDS)
mask=mask.astype(bool).flatten()

In [ ]:
print(mask)

In [ ]:
print(len(img1pts))
print(len(img2pts))
print(len(mask))

In [ ]:
#Inliers // Optional
img1pts = img1pts[mask==True]
img2pts = img2pts[mask==True]
mask = len(img1pts) * [True] ### We need the match matrix to be the same size of the number of points

In [ ]:
print(len(img1pts))
print(len(img2pts))
print(len(mask))

### Compute Epipolar Lines

In [ ]:
def ComputeEpiline(pts, index, F):
    if pts.shape[1]==2:
        #converting to homogenous coordinates if not already
        pts = cv2.convertPointsToHomogeneous(pts)[:,0,:]

    if index==1:
        lines = F.dot(pts.T)
    elif index==2:
        lines = F.T.dot(pts.T)

    return lines.T

In [ ]:
lines2=ComputeEpiline(img1pts[mask],1,F)
lines1=ComputeEpiline(img2pts[mask],2,F)

In [ ]:
def drawlines(img1,img2,lines,pts1,pts2,drawOnly=None,linesize=7,circlesize=10):
    r,c = img1.shape[:-1]

    img1_, img2_ = np.copy(img1), np.copy(img2)

    drawOnly = lines.shape[0] if (drawOnly is None) else drawOnly

    i = 0
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])

        img1_ = cv2.line(img1_, (x0,y0), (x1,y1), color,linesize)
        img1_ = cv2.circle(img1_,tuple(pt1.astype(int)),circlesize,color,-1)
        img2_ = cv2.circle(img2_,tuple(pt2.astype(int)),circlesize,color,-1)

        i += 1

        if i > drawOnly:
            break

    return img1_,img2_

In [ ]:
epilines1, epilines2 = drawlines(img2,img1,lines2,img2pts[mask],img1pts[mask],drawOnly=10,linesize=18,circlesize=10)
epilines3, epilines4 = drawlines(img1,img2,lines1,img1pts[mask],img2pts[mask],drawOnly=10,linesize=18,circlesize=10)

epilines12 = np.concatenate((epilines2, epilines1), axis=1)
plt.imshow(epilines12)
plt.show()

epilines34 = np.concatenate((epilines3, epilines4), axis=1)
plt.imshow(epilines34)
plt.show()

epilines = np.concatenate((epilines3, epilines1), axis=1)

plt.imshow(epilines)
plt.show()

### 4. Essential Matrix

In [ ]:
img1pts[mask]

ref: https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga0b166d41926a7793ab1c351dbaa9ffd4

In [ ]:
K = np.array([[2759.48,0,1520.69],[0,2764.16,1006.81],[0,0,1]])
E = cv2.findEssentialMat(img1pts[mask],
                         img2pts[mask],
                         K,
                         method=cv2.RANSAC,
                         prob=0.95,
                         threshold=1e-3,
                         maxIters=1000)
E = E[0]

In [ ]:
# E = K.T @ F @ K

In [ ]:
print(E)

### 5. Camera Poses

In [ ]:
pts_rec, r_rec, t_rec, mask_rec = cv2.recoverPose(E, img1pts, img2pts)

### 6. Triangulation

In [ ]:
def GetTriangulatedPts(img1pts,img2pts,K,R,t):
    img1ptsHom = cv2.convertPointsToHomogeneous(img1pts)[:,0,:]
    img2ptsHom = cv2.convertPointsToHomogeneous(img2pts)[:,0,:]

    img1ptsNorm = (np.linalg.inv(K).dot(img1ptsHom.T)).T
    img2ptsNorm = (np.linalg.inv(K).dot(img2ptsHom.T)).T

    img1ptsNorm = cv2.convertPointsFromHomogeneous(img1ptsNorm)[:,0,:]
    img2ptsNorm = cv2.convertPointsFromHomogeneous(img2ptsNorm)[:,0,:]

    pts4d = cv2.triangulatePoints(np.eye(3,4),np.hstack((R,t)),img1ptsNorm.T,img2ptsNorm.T)
    pts3d = cv2.convertPointsFromHomogeneous(pts4d.T)[:,0,:]

    return pts3d

In [ ]:
# R1, R2, t = cv2.decomposeEssentialMat(E)

In [ ]:
pts3d = GetTriangulatedPts(img1pts,img2pts,K,r_rec, t_rec)

### 7. Save PLY

In [ ]:
def pts2ply(pts,filename='out.ply'):
    f = open(filename,'w')
    f.write('ply\n')
    f.write('format ascii 1.0\n')
    f.write('element vertex {}\n'.format(pts.shape[0]))

    f.write('property float x\n')
    f.write('property float y\n')
    f.write('property float z\n')

    f.write('property uchar red\n')
    f.write('property uchar green\n')
    f.write('property uchar blue\n')

    f.write('end_header\n')

    for pt in pts:
        f.write('{} {} {} 255 255 255\n'.format(pt[0],pt[1],pt[2]))
    f.close()

In [ ]:
pts2ply(pts3d)

### 9. Try Loop

In [ ]:
import collections
topologies = collections.OrderedDict()
topologies['360'] = tuple(zip((0,1,2,3,4,5,6,7,8,9,10,11),
                          (1,2,3,4,5,6,7,8,9,10,11,0)))

topologies['overlapping'] = tuple(zip((0,1,2,3,4,5,6,7,8,9),
                          (1,2,3,4,5,6,7,8,9,10)))

topologies['adjacent'] = tuple(zip((0,2,4,6,8,10),
                     (1,3,5,7,9,11)))

topologies['skipping_1'] = tuple(zip((0,3,6,9),
                 (1,4,7,10)))

topologies['skipping_2'] = tuple(zip((0,4,8),
                 (1,5,9)))

topologies["zero"] = tuple(zip((0,0,0),
                 (1,1,1)))

In [ ]:
import glob
import os

# os.rename("fountain/00010.png","fountain/0010.png")
# os.rename("fountain/00011.png","fountain/0011.png")
images= sorted(glob.glob("fountain/*.png"))

print(images)

images_cv = [cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB) for img in images]

In [ ]:
def main(K, images_cv, topology):
    xyz_global_array = [None]*len(topology)
    for pair_index, (left_index,right_index) in enumerate(topology):
        print(pair_index)
        img1 = images_cv[left_index]
        img2 = images_cv[right_index]

        # 1. Feature Matching
        kp1,desc1,kp2,desc2,matches=GetImageMatches(img1,img2)
        img1pts,img2pts,img1idx,img2idx=GetAlignedMatches(kp1,desc1,kp2,desc2,matches)

        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                    singlePointColor = None,
                    flags = 2)

        plt.imshow(cv2.drawMatches(img1,kp1,img2,kp2,matches,None,**draw_params))
        plt.show()

        #2. Fundamental
        F, mask = cv2.findFundamentalMat(img1pts, img2pts, method=cv2.FM_7POINT)
        mask=mask.astype(bool).flatten()

        #2.2 Inliers // Optional
        img1pts = img1pts[mask==True]
        img2pts = img2pts[mask==True]
        mask = len(img1pts) * [True] ### We need the match matrix to be the same size of the number of points

        #3. Essential
        E = K.T.dot(F.dot(K))

        #4. R, T
        #R1,R2,t = ExtractCameraPoses(E)
        #t = t[:,np.newaxis]
        pts_rec, r_rec, t_rec, mask_rec = cv2.recoverPose(E, img1pts, img2pts)

        #5. Triangulate
        #pts3d = GetTriangulatedPts(img1pts[mask],img2pts[mask],K,R2,t)
        pts3d = GetTriangulatedPts(img1pts[mask],img2pts[mask],K,r_rec,t_rec)

        #6. Add to Global Points
        xyz_global_array[pair_index] = pts3d

    return xyz_global_array

full_pts3d = main(K, images_cv, topologies["overlapping"])

In [ ]:
for idx, points in enumerate(full_pts3d):
    pts2ply(full_pts3d[idx], "out_1_{}.ply".format(idx))

In [ ]:
xyz = np.vstack(full_pts3d)
pts2ply(xyz, "out_full.ply")